# Turning simulation output into time-averaged profiles
AFiD outputs regular plane-averaged profiles $f(x,t)$ into the file `outputdir/means.h5` inside each simulation directory. This notebook loops over the simulations to combine the data into two databases of time-averaged profiles $\overline{f}(x)$, containing data for all the simulations. These files are the CSV formatted `base_profiles.csv` and `ref_profiles.csv` which store data for variables on the coarse and refined grid respectively.

In [1]:
# Begin by loading the required pacakages
import h5py
import afidtools as afid
import numpy as np
import pandas as pd

In [2]:
# Create the list of simulation directories
flist = [
    "Sc10/Pr1", "Sc10/Pr2", "Sc10/Pr5", "Sc10/Pr20", "Sc10/Pr50", "Sc10/Pr100",
    "Sc100/Pr1", "Sc100/Pr2", "Sc100/Pr5", "Sc100/Pr10", "passive/Sc10", "passive/Sc100"
]

In [3]:
# Collect wall-normal coordinate grid from each simulation,
# for both coarse and refined grids, and store in a list
xlist, xrlist = [], []
for f in flist:
    xlist.append(afid.Grid(f).xm)
    xrlist.append(afid.Grid(f).xmr)

In [4]:
# Collect coordinates in time of the time series for each simulation
tlist = []
for f in flist:
    tlist.append(np.array(afid.mean_time(f)))

In [5]:
# Specify when to start averaging over each simulation
# N.B. These are chosen to avoid initial transient effects
t0list = [
    100, 400, 400, 900, 360, 450,
    650, 400, 420, 400, 100, 860
]

In [6]:
# Construct a list of all the variables output by the simulations
# These are explicitly defined in the documentation at
# https://chowland.github.io/AFiD-MuRPhFi/mean_outs/
with h5py.File(flist[0]+"/outputdir/means.h5","r") as f:
    varlist = list(f.keys())
    print(varlist)

['Sbar', 'Srms', 'Tbar', 'Trms', 'chiS', 'chiT', 'epsilon', 'vxS', 'vxT', 'vxrms', 'vxvy', 'vxvz', 'vyS', 'vyT', 'vybar', 'vyrms', 'vzS', 'vzT', 'vzbar', 'vzrms']


In [7]:
# Initialise lists to fill with DataFrames from each simulation
dlst, drlst = [], []

# Loop over the simulation directories
for i, fld in enumerate(flist):
    # Load the grid coordinates and time coordinates
    x = xlist[i]
    xr = xrlist[i]
    t = tlist[i]
    t0 = t0list[i]
    
    # Read physical input parameters from the simulation input file
    inputs = afid.InputParams(fld)
    RaT, RaS = inputs.active_T*inputs.RayT, inputs.RayS
    Pr, Sc = inputs.PraT, inputs.PraS
    
    # Start building DataFrames with physical parameters and coordinates
    df = pd.DataFrame(
        {
            "RaT": RaT, "Pr": Pr,
            "RaS": RaS, "Sc": Sc,
            "x": x[:x.size//2]
        }
    )
    dfr = pd.DataFrame(
        {
            "RaT": RaT, "Pr": Pr,
            "RaS": RaS, "Sc": Sc,
            "x": xr[:xr.size//2]
        }
    )
    
    # Loop over variables to record
    for var in varlist:
        # Read array for times we want to average over
        tmp = afid.read_mean(fld, var)[:,t > t0]
        
        # Simulation stores rms values including the mean.
        # We want squared perturbation quantities, so we
        # square the rms and subtract the square of the mean
        if var in ["vyrms", "vzrms", "Trms", "Srms"]:
            tmp = tmp**2 - (afid.read_mean(fld, var[:-3]+"bar")[:,t > t0])**2
        # By definition vxbar==0, so no need to subtract a mean
        elif var=="vxrms":
            tmp = tmp**2
        # Change the name of the rms variables in the DataFrame
        # to reflect these are squared perturbations now
        if "rms" in var:
            vname = var[:-3]+"p2"
        else:
            vname = var
        
        # Through the problem setup, all mean profiles are anti-symmetric
        # and all second-order quantities are symmetric.
        # We take advantage of this, and average the two halves of
        # the domain together to gain more statistics
        
        # Build arrays taking advantage of the symmetry
        n = tmp.shape[0]
        if "bar" in var:
            tmp = np.concatenate((tmp[:n//2,:], -np.flip(tmp[n//2:,:])), axis=1)
        else:
            tmp = np.concatenate((tmp[:n//2,:], np.flip(tmp[n//2:,:])), axis=1)
        
        # Record mean and standard deviation (with respect to time)
        if n==x.size:
            df[vname] = tmp.mean(axis=1)
            df["σ"+vname] = tmp.std(axis=1)
        elif n==xr.size:
            dfr[vname] = tmp.mean(axis=1)
            dfr["σ"+vname] = tmp.std(axis=1)
    
    # Append DataFrames from each individual simulation to the lists
    dlst.append(df)
    drlst.append(dfr)

# Construct DataFrames covering all simulations by putting all data together
df = pd.concat(dlst)
dfr = pd.concat(drlst)

In [8]:
# Add in a density ratio variable to the DataFrames
df["Rρ"] = -df["RaT"]*df["Sc"]/df["RaS"]/df["Pr"]
dfr["Rρ"] = -dfr["RaT"]*dfr["Sc"]/dfr["RaS"]/dfr["Pr"]
df.Rρ.unique()

array([2.e-02, 5.e+01, 0.e+00])

In [9]:
# Add (dimensionless) diffusivity coefficients to the DataFrames
# (noting that for high Pr sims, we have switched T and S for convenience)
df["ν"] = ( ((df["Rρ"]==50)*(df["Pr"]/df["RaT"])**0.5).fillna(0) + ((df["Rρ"]!=50).fillna(0)*(df["Sc"]/df["RaS"])**0.5).fillna(0) )
df["κT"] = df["ν"]/df["Pr"]
df["κS"] = df["ν"]/df["Sc"]
dfr["ν"] = ( ((dfr["Rρ"]==50)*(dfr["Pr"]/dfr["RaT"])**0.5).fillna(0) + ((dfr["Rρ"]!=50).fillna(0)*(dfr["Sc"]/dfr["RaS"])**0.5).fillna(0) )
dfr["κT"] = dfr["ν"]/dfr["Pr"]
dfr["κS"] = dfr["ν"]/dfr["Sc"]

In [10]:
# Preview the DataFrames
df.head()

,RaT,Pr,RaS,Sc,x,Tbar,σTbar,Tp2,σTp2,chiT,...,vzT,σvzT,vzbar,σvzbar,vzp2,σvzp2,Rρ,ν,κT,κS
0,-20000.0,1.0,10000000.0,10.0,0.000688,-0.496809,0.000200,0.000003,6.313947e-07,0.028058,...,-0.000008,0.000053,0.000016,0.000107,0.000006,0.000001,0.02,0.001,0.001,0.0001
1,-20000.0,1.0,10000000.0,10.0,0.002117,-0.490184,0.000614,0.000029,5.973520e-06,0.028057,...,-0.000024,0.000161,0.000050,0.000330,0.000053,0.000012,0.02,0.001,0.001,0.0001
2,-20000.0,1.0,10000000.0,10.0,0.003651,-0.483076,0.001057,0.000086,1.775150e-05,0.028047,...,-0.000041,0.000272,0.000085,0.000568,0.000153,0.000034,0.02,0.001,0.001,0.0001
3,-20000.0,1.0,10000000.0,10.0,0.005288,-0.475489,0.001531,0.000181,3.720458e-05,0.028018,...,-0.000058,0.000386,0.000124,0.000823,0.000310,0.000068,0.02,0.001,0.001,0.0001
4,-20000.0,1.0,10000000.0,10.0,0.007029,-0.467427,0.002032,0.000320,6.557969e-05,0.027955,...,-0.000076,0.000502,0.000164,0.001093,0.000528,0.000115,0.02,0.001,0.001,0.0001


In [11]:
dfr.head()

,RaT,Pr,RaS,Sc,x,Sbar,σSbar,Sp2,σSp2,chiS,...,vxS,σvxS,vyS,σvyS,vzS,σvzS,Rρ,ν,κT,κS
0,-20000.0,1.0,10000000.0,10.0,0.000192,-0.497587,0.000100,0.000001,2.160941e-07,0.018790,...,-1.378325e-09,4.226866e-10,-0.002868,0.000145,-0.000005,0.000035,0.02,0.001,0.001,0.0001
1,-20000.0,1.0,10000000.0,10.0,0.000592,-0.492577,0.000308,0.000011,2.045367e-06,0.018789,...,-1.635874e-08,4.693058e-09,-0.004093,0.000208,-0.000007,0.000049,0.02,0.001,0.001,0.0001
2,-20000.0,1.0,10000000.0,10.0,0.001021,-0.487198,0.000531,0.000032,6.083622e-06,0.018788,...,-6.180830e-08,1.767146e-08,-0.006160,0.000318,-0.000011,0.000075,0.02,0.001,0.001,0.0001
3,-20000.0,1.0,10000000.0,10.0,0.001479,-0.481450,0.000770,0.000066,1.276973e-05,0.018784,...,-1.671903e-07,4.772083e-08,-0.008867,0.000465,-0.000016,0.000109,0.02,0.001,0.001,0.0001
4,-20000.0,1.0,10000000.0,10.0,0.001966,-0.475333,0.001023,0.000117,2.256594e-05,0.018775,...,-3.758897e-07,1.070555e-07,-0.011744,0.000625,-0.000022,0.000145,0.02,0.001,0.001,0.0001


In [12]:
# Write the DataFrames to CSV files
df.to_csv("base_profiles.csv")
dfr.to_csv("ref_profiles.csv")